In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import cv2
import gymnasium as gym
from gymnasium import Env
from gymnasium import spaces
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from gymnasium.spaces import Box, MultiBinary
import pygetwindow as gw
import mss.tools
import mss
import numpy as np
import pyautogui
import time
import keyboard

In [ ]:
gw.getAllTitles()

In [53]:
import pygetwindow as gw
import mss

# Function to capture a specific window by title and return a processed image
def capture_window(window_title):
    # Find the window by title
    window_list = gw.getWindowsWithTitle(window_title)
    if not window_list:
        print(f'Window with title "{window_title}" not found!')
        return None

    window = window_list[0]

    # Get the window's position and size
    left, top, right, bottom = window.left, window.top, window.right, window.bottom
    width, height = right - left, bottom - top

    # Adjust the capture region
    monitor = {"top": top, "left": left, "width": width, "height": height}

    # Grab the data using mss
    h_ight = 617
    h_ight2 = 568
    point_positions = [(410, h_ight), (494, h_ight), (581, h_ight), (666, h_ight)]
    with mss.mss() as sct:
        sct_img =sct.grab(monitor)
        greyscale_img = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
        #resized_img = cv2.resize(greyscale_img, (64, 64), interpolation=cv2.INTER_AREA)
        pixel_values = [greyscale_img[y, x] for x, y in point_positions]
    return np.array(pixel_values, dtype=np.uint8)

# Example usage
#img = capture_window('osu!')7

In [54]:
class OsuManiaEnv(Env):
    
    def __init__(self, test_mode=False):
        self.test_mode = test_mode
        self.key_states = [0, 0, 0, 0]
        super(OsuManiaEnv, self).__init__()
        self.action_space = MultiBinary(4)
        self.sim_len = 4000 if not self.test_mode else 100000
        self.state = self._get_screen_image()
        self.h_ight = 51
        #self.point_positions = [(7, self.h_ight), (23, self.h_ight), (40, self.h_ight), (55, self.h_ight)]
        self.keys = ['d', 'f', 'j', 'k']
        self.truncated = False
        self.info = {}
        self.observation_space = spaces.Box(low=0, high=255, shape=(4,), dtype=np.uint8)
        keyboard.press('shift')
        time.sleep(0.6)
        keyboard.release('shift')
        time.sleep(1)

    def reset(self, seed = None):
        # Reset the environment state
        self.sim_len = 4000 if not self.test_mode else 100000
        keyboard.press('shift')
        time.sleep(0.6)
        keyboard.release('shift')
        time.sleep(1)
        self.state = self._get_screen_image()
        self.key_states = [0, 0, 0, 0]
        for i in self.keys:
             keyboard.release(i)
        return self.state, self.info

    def step(self, action):
        self._perform_action(action)
        self.state = self._get_screen_image()
        reward = self._calculate_reward(action, self.state)
        terminated = self.sim_len <= 0
        self.sim_len -= 1
        return self.state, reward, terminated, self.truncated, self.info 

    def _get_screen_image(self):
        return capture_window('osu!')

    def _perform_action(self, action):
        for idx, act in enumerate(action):
            if act == 1 and self.key_states[idx] == 0:
                keyboard.press(self.keys[idx])
                self.key_states[idx] = 1
            elif act == 0 and self.key_states[idx] == 1:
                keyboard.release(self.keys[idx])
                self.key_states[idx] = 0

    def _calculate_reward(self, action, img):
        reward = 0
        white_threshold = 200
    
        for idx, pixel_value in enumerate(img[:4]):
            # Check if the action is to press the key
            if action[idx] == 1:
                if pixel_value > white_threshold:
                    # Reward if the pixel value is white and the key is pressed
                    reward += 4
                else:
                    # Penalize if the key is pressed but the pixel is not white
                    reward -= 2
            else:
                if pixel_value > white_threshold:
                    # Penalize if the key is not pressed but the pixel is white
                    reward -= 2
        #print(img, action, reward)
        return reward

    def _check_done(self):
        # Placeholder for done condition logic
        return False

env = OsuManiaEnv()
#from stable_baselines3.common.env_checker import check_env
#check_env(env)

In [55]:
#import os
#log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, device='cuda')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [39]:
env = OsuManiaEnv()
vec_env = DummyVecEnv([lambda: env])
model.set_env(vec_env)

In [ ]:
time.sleep(2)
model.learn(total_timesteps=100000000)

-----------------------------
| time/              |      |
|    fps             | 67   |
|    iterations      | 1    |
|    time_elapsed    | 30   |
|    total_timesteps | 2048 |
-----------------------------


KeyboardInterrupt: 

In [338]:
#model.save("./model2s/")

In [12]:
model = PPO.load("./models/", device='cuda')

In [7]:
from stable_baselines3.common.vec_env import DummyVecEnv
model = PPO.load("./models/", device='cuda')
env = OsuManiaEnv()
vec_env = DummyVecEnv([lambda: env])
model.set_env(vec_env)
time.sleep(2)
model.learn(total_timesteps=100000000, reset_num_timesteps=False, log_interval=500)

KeyboardInterrupt: 

In [391]:
# Number of episodes to test
env = OsuManiaEnv(test_mode=True)
n_episodes = 1
time.sleep(2)

for episode in range(n_episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        # Use the model to predict the next action
        action, _states = model.predict(obs, deterministic=True)
        # Take the action in the environment
        obs, reward, done, done2, info = env.step(action)        
        total_reward += reward
    
    print(f"Episode {episode + 1} reward: {total_reward}")

KeyboardInterrupt: 

In [37]:
import pygetwindow as gw
import dxcam
import cv2

# Initialize the camera
camera = dxcam.create()

def capture_and_save_window(window_title, save_path):
    # Find the window by title
    window_list = gw.getWindowsWithTitle(window_title)
    if not window_list:
        print(f'Window with title "{window_title}" not found!')
        return

    window = window_list[0]

    # Get the window's position and size
    left, top, right, bottom = window.left, window.top, window.right, window.bottom
    width, height = right - left, bottom - top

    # Define the region to capture
    region = (left, top, right, bottom)

    # Capture the screen region
    frame = camera.grab(region)
    
    if frame is None:
        print("Failed to capture screen.")
        return

    # Save the captured frame
    cv2.imwrite(save_path, frame)
    print(f'Screenshot saved to {save_path}')

# Example usage
window_title = "osu!"
save_path = "osu_screenshot.png"
capture_and_save_window(window_title, save_path)


You already created a DXCamera Instance for Device 0--Output 0!
Returning the existed instance...
To change capture parameters you can manually delete the old object using `del obj`.
Screenshot saved to osu_screenshot.png


In [196]:
time.sleep(3)
pyautogui.keyDown('shift')
time.sleep(0.6)
pyautogui.keyUp('shift')
time.sleep(2)

In [25]:
time.sleep(3)
pyautogui.keyDown('shift')
time.sleep(0.6)
pyautogui.keyUp('shift')

In [230]:
import pygetwindow as gw
import cv2
import mss
import mss.tools
import numpy as np
import time

# Function to capture a specific window by title and save as PNG
def capture_window_save_img(window_title):
    # Find the window by title
    window_list = gw.getWindowsWithTitle(window_title)
    if not window_list:
        print(f'Window with title "{window_title}" not found!')
        return

    window = window_list[0]

    # Get the window's position and size
    left, top, right, bottom = window.left, window.top, window.right, window.bottom
    width, height = right - left, bottom - top

    # Adjust the capture region (example: exclude borders)
    monitor = {"top": top+544, "left": left+373, "width": width-373-107, "height": height-555}
    output = f"sct-{monitor['top']}x{monitor['left']}_{monitor['width']}x{monitor['height']}.png"

    # Grab the data using mss
    with mss.mss() as sct:
        sct_img = sct.grab(monitor)

        # Convert to numpy array
        img = np.array(sct_img)

        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Resize to 64x64
        resized_img = cv2.resize(gray_img, (64, 64), interpolation=cv2.INTER_AREA)

        # Save the resized grayscale image
        cv2.imwrite(output, resized_img)
        print(f"Resized grayscale screenshot saved to {output}")

# Example usage:
time.sleep(3)
capture_window_save_img('osu!')


Resized grayscale screenshot saved to sct-850x1416_326x85.png


In [ ]:
import cv2
import numpy as np

# Define the positions of the points you want to monitor (example positions)
point_positions = [(7, 54), (23, 54), (40, 54), (55, 54)]

# Function to calculate reward based on points being white or black
def calculate_reward(img):
    reward = 0
    # Check each point position
    for pos in point_positions:
        x, y = pos
        pixel_value = img[y, x]  # Access pixel value at (x, y)
        # Adjust condition based on actual values (assuming white is 255 and black is 0)
        if pixel_value > 200:  # Change threshold based on your image characteristics
            reward += 2  # Increment reward if point is white
        else:
            reward -= 1  # Decrement reward if point is black
        print(pos, pixel_value, reward) 

    return reward

# Example usage:
# Load an image, ensure it is grayscale
image_path = 'example.png'  # Replace with your image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale

if image is None:
    print(f"Failed to load image from {image_path}")
else:
    # Check image size and process
    if image.shape == (64, 64):  # Ensure the image is of the expected size
        reward = calculate_reward(image)
        print(f"Reward calculated: {reward}")
    else:
        print(f"Unexpected image dimensions: {image.shape}")


In [200]:
class OsuManiaEnv(Env):
    
    def __init__(self):
        self.key_states = [0, 0, 0, 0]
        super(OsuManiaEnv, self).__init__()
        self.action_space = MultiBinary(4)
        self.sim_len = 150;
        self.state = self._get_screen_image()
        self.h_ight = 51
        self.point_positions = [(7, self.h_ight), (23, self.h_ight), (40, self.h_ight), (55, self.h_ight)]
        self.keys = ['d', 'f', 'j', 'k']
        self.truncated = False
        self.info = {}
        self.observation_space = MultiBinary(4)
        pyautogui.keyDown('shift')
        time.sleep(0.6)
        pyautogui.keyUp('shift')
        time.sleep(2)

    def reset(self, seed = None):
        # Reset the environment state
        self.sim_len = 100;
        for i in self.keys:
            pyautogui.keyUp(i)
        pyautogui.keyDown('shift')
        time.sleep(0.6)
        pyautogui.keyUp('shift')
        time.sleep(2)
        self.state = self._get_screen_image()
        self.key_states = [0, 0, 0, 0]
        return self.state, {}

    def step(self, action):
        self._perform_action(action)
        self.state = self._get_screen_image()
        reward = self._calculate_reward(action, self.state)  # Define your reward function

        terminated = self.sim_len <= 0
        self.sim_len -= 1
        return self.state, reward, terminated, self.truncated, self.info 

    def _get_screen_image(self):
        return capture_window('osu!')

    def _perform_action(self, action):
        for idx, act in enumerate(action):
            if act == 1 and self.key_states[idx] == 0:
                pyautogui.keyDown(self.keys[idx])
                self.key_states[idx] = 1
            elif act == 0 and self.key_states[idx] == 1:
                pyautogui.keyUp(self.keys[idx])
                self.key_states[idx] = 0

    def _calculate_reward(self, action, img):
        reward = 0
        # Check each point position
        for idx, pos in  enumerate(self.point_positions):
            x, y = pos
            pixel_value = img[0, x, y]  # Access pixel value at (x, y)
            # Adjust condition based on actual values (assuming white is 255 and black is 0)
            if pixel_value > 100:  # Change threshold based on your image characteristics
                if action[idx] == 1:
                    reward += 2
                else:
                    reward -= 1
            else:
                if action[idx] == 1:
                    reward -=1
                else:
                    reward = 0
        #print(pix_arr, action, reward)
        #print(self.sim_len)
        return reward

env = OsuManiaEnv()
from stable_baselines3.common.env_checker import check_env
check_env(env)

AssertionError: The observation returned by the `reset()` method does not match the data type (cannot cast) of the given observation space MultiBinary(4). Expected: int8, actual dtype: uint8

In [530]:
def capture_window(window_title):
    window_list = gw.getWindowsWithTitle(window_title)
    if not window_list:
        print(f'Window with title "{window_title}" not found!')
        return None

    window = window_list[0]
    left, top, right, bottom = window.left, window.top, window.right, window.bottom
    width, height = right - left, bottom - top
    monitor = {"top": top + 544, "left": left + 373, "width": width - 373 - 107, "height": height - 555}

    with mss.mss() as sct:
        sct_img = sct.grab(monitor)
        img = np.array(sct_img, dtype=np.uint8)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(gray_img, (64, 64), interpolation=cv2.INTER_AREA)
        final_img = resized_img / 255.0

    return np.array([final_img])

class OsuManiaEnv(Env):
    
    def __init__(self):
        self.key_states = [0, 0, 0, 0]
        super(OsuManiaEnv, self).__init__()
        self.action_space = MultiBinary(4)
        self.sim_len = 150
        self.state = self._get_screen_image()
        self.h_ight = 51
        self.point_positions = [(7, self.h_ight), (23, self.h_ight), (40, self.h_ight), (55, self.h_ight)]
        self.keys = ['d', 'f', 'j', 'k']
        self.truncated = False
        self.info = {}
        self.observation_space = spaces.Box(low=0, high=255, shape=(1, 64, 64), dtype=np.uint8)
        pyautogui.keyDown('shift')
        time.sleep(0.6)
        pyautogui.keyUp('shift')
        time.sleep(2)

    def reset(self, seed=None):
        self.sim_len = 150
        for key in self.keys:
            pyautogui.keyUp(key)
        pyautogui.keyDown('shift')
        time.sleep(0.6)
        pyautogui.keyUp('shift')
        time.sleep(2)
        self.state = self._get_screen_image()
        self.key_states = [0, 0, 0, 0]
        return self.state, {}

    def step(self, action):
        self._perform_action(action)
        self.state = self._get_screen_image()
        reward = self._calculate_reward(action, self.state)
        terminated = self.sim_len <= 0
        self.sim_len -= 1
        return self.state, reward, terminated, self.truncated, self.info

    def _get_screen_image(self):
        return capture_window('osu!')

    def _perform_action(self, action):
        for idx, act in enumerate(action):
            if act == 1 and self.key_states[idx] == 0:
                pyautogui.keyDown(self.keys[idx])
                self.key_states[idx] = 1
            elif act == 0 and self.key_states[idx] == 1:
                pyautogui.keyUp(self.keys[idx])
                self.key_states[idx] = 0

    def _calculate_reward(self, action, img):
        reward = 0
        for idx, pos in enumerate(self.point_positions):
            x, y = pos
            pixel_value = img[0, x, y]
            if pixel_value > 0.5:
                if action[idx] == 1:
                    reward += 2
                else:
                    reward -= 1
            else:
                if action[idx] == 1:
                    reward -= 1
                else:
                    reward += 1
        return reward

env = OsuManiaEnv()